In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import pickle
from sklearn import preprocessing

In [ ]:
clfs = {}
ys = {}
fexp = {}
XC = {}
for OLD_YEARS in [True,False]:
    with open('clf_{}.pkl'.format(OLD_YEARS),'rb') as fp:
        clfs[OLD_YEARS] = pickle.load(fp)
    with open('ys_{}.pkl'.format(OLD_YEARS),'rb') as fp:
        ys[OLD_YEARS] = pickle.load(fp)
    with open('fexp_{}.pkl'.format(OLD_YEARS),'rb') as fp:
        fexp[OLD_YEARS] = pickle.load(fp)
    with open('XC_{}.pkl'.format(OLD_YEARS),'rb') as fp:
        XC[OLD_YEARS] = pickle.load(fp)


In [ ]:
all_tables = {}
for ty in range(1952,2020): 
    all_tables[ty] = np.load('tables_{}.pkl'.format(ty))


In [ ]:
tyear = list(range(1952,2020))


In [ ]:
table_columns = defaultdict(dict)
for year in all_tables:
    for team in all_tables[year]:
        for table_name in all_tables[year][team]:
            if table_name in ['injury','playoffs_pbp','pbp','on_off','on_off_p','draft-rights','team_and_opponent','conf','name','logo','roster','contracts','salaries2']:
                continue
            table = all_tables[year][team][table_name].replace('',0).fillna(0)
            try:
                table = table.drop('Team Totals')
            except:
                pass
            W = table.MP
            table_columns[(year,team)][table_name] = table.columns
            table = pd.DataFrame([np.average(table,0,W)],columns=table.columns)
            all_tables[year][team][table_name] = table


In [ ]:
table_mask = defaultdict(dict)
for key in table_columns:
    for table in table_columns[key]:
        table_mask[key][table] = [_.strip() !=''  for _ in table_columns[key][table] ]
        table_columns[key][table] = [_ for _ in table_columns[key][table] if _.strip() != '']
#for player in player_stats:
#    for table_in in player_stats[player]:
#        if 'on_off' in table_in or 'salaries' in table_in:
#            continue
#        if len(player_stats[player][table_in]) > 1:
#            pass
            #print(player,table_in,'MP' in player_stats[player][table_in][0]._fields)
            #print(player_stats[player][table_in][0])

In [ ]:
all_tables[2019]['ATL']['per_minute']


In [ ]:
lt = [('FG%','totals'),('MP','per_game'),('Ht','roster'),('Birth Date','roster'),
            ('FG','per_game'),('FGA','per_game'),('3P','per_game'),('3PA','per_game'),('3P%','totals'),
            ('FT','per_game'),('ORB','per_game'),('TRB','per_game'),('FT%','totals'),
            ('FTA','per_game'),('DRB','per_game'),('AST','per_game'),('2P','per_game'),('2PA','per_game'),
            ('STL','per_game'),('TOV','per_game'),('Blk','per_game','BLK'),('PER','advanced'),
            ('PF','per_game'),('PTS','per_game','PTS/G'),('PER','advanced'),('PTS','per_game','PTS'),
            ('OPM','advanced','OBPM'),('BPM','advanced'),('USG%','advanced'),('DPM','advanced','DBPM'),
            #('AtRimFGP','totals','2P%'), ('LowPostFGP','totals','2P%'), ('MidRangeFGP','totals','eFG%'),
            ('+/-','advanced','BPM'),# ('AtRimFGP','shooting','2P%'), ('LowPostFGP','shooting','2P%'),
            ('3PAr','advanced'),('FTr','advanced'),('MPT','totals','MP')]

locations = []
for loc in lt:
    if len(loc) == 2:
        loc = list(loc)
        loc.append(loc[0])
    locations.append(loc)
    
for loc in locations:
    if loc[1] == 'per_game':
        locations.append((loc[0] + 'p36','per_minute',loc[2]))
        locations.append((loc[0] + 'p100','per_poss',loc[2]))
        
locs = defaultdict(dict)
for ty in tyear:
    for team in all_tables[ty]:
        for loc in locations:
            try:
                locs[(ty,team)][loc[0]] = (table_columns[(ty,team)][loc[1]].index(loc[2]),loc[1])
            except Exception as e:
                #print(e)
                #print(team,ty)
                pass#print(loc,table_columns[loc[1]])
    #if 'shooting' in table_columns:
    #    l = list(locs['LowPostFGP'])
    #    l[0] +=2
    #    locs['MidRangeFGP'] = l
#locs

In [ ]:
player_vectors = []
player_names = []
player_teams = []
player_years = []
player_scales = []
player_heights = []
for ty in tyear:
    for name in all_tables[ty]:
        try:
            stats = all_tables[ty][name]
            team = name
            d = {}
            for k,v in locs[(ty,name)].items():
                try:
                    SV = stats[v[1]].iloc[0,v[0]]
                    d[k] = SV
                except:
                    print(k,v)
                    #print(k,v)
                    print(stats)
                    raise
            try:
                ht = [int(_) for _ in d['Ht'].split('-')]
                hgt = ht[0]*12 + ht[1]
            except:
                hgt=70
            d['Hgt'] = 3.7*(hgt-66)

            try:# 'roster' in stats:
                d['Age']= stats['totals'].loc[0,'Age']
            except:
                d['Age']=np.random.randint(25,35)

            if ty < 1980 and '3Pp36' in d:
                d['PTSp36'] -= d['3Pp36']

            if '3PAp100' in d:
                d['DRE'] = -8.4+ 0.8*d['PTSp100'] -0.7*(d['FGAp100']-d['3PAp100']) -0.6*d['3PAp100'] - 0.2*d['FTp100'] + 0.1*d['ORBp100'] + 0.4*d['DRBp100'] + 0.5*d['ASTp100'] + 1.7*d['STLp100'] + 0.8*d['Blkp100'] - 1.4*d['TOVp100'] - 0.1*d['PFp100']
                d['3PtP'] = (2/(1+np.exp(-d['3PAp100']))-1)*d['3P%']
                d['Creation'] = d['ASTp100']*0.1843+(d['PTSp100']+d['TOVp100'])*0.0969-2.3021*d['3PtP']+0.0582*(d['ASTp100']*(d['PTSp100']+d['TOVp100'])*d['3PtP'] )-1.1942
                d['Load'] = (d['ASTp100']-(0.38*d['Creation'])*0.75)+d['FGAp100']+d['FTAp100']*0.44+d['Creation']+d['TOVp100']
                d['cTOV'] = d['TOVp100']/d['Load']
                d['DPM'] = d['Blkp100']*0.802+d['DRBp100']*0.42-4.7-0.07551*d['PFp100']+1.597019*d['STLp100']-0.26385*d['TOVp100']
                d['OPM'] = -8.57647+0.6111*d['PTSp100']-0.33918*(0.44*d['FTAp100']+d['FGAp100'])+0.440814*d['FTAp100']+0.379745*d['3PAp100']+0.634044*d['ASTp100']+0.77827*d['ORBp100']-1.08855*d['TOVp100']+0.26262*d['STLp100']
                d['BPM'] = d['OPM']+d['DPM']
                Xcol = XC[False]
            else:
                Xcol = XC[True]

            d['DREO'] = d['PTSp36'] + .2*d['TRBp36'] + .5*d['ASTp36'] - .9*d['FGAp36'] - .35*d['FTAp36']-6.23
            d['OSPM']= d['PTSp36']*0.15 +  d['FTAp36']*0.03 +  d['ASTp36']*0.12 +  d['TRBp36']*0.09 \
                      -d['FTp36']*0.10 -  d['FGAp36']*0.06 -d['FGp36']*0.24 -0.92

            MP = np.maximum(1,np.nan_to_num(d['MPT']))
            player_vectors.append([d[stat] for stat in Xcol])
            player_scales.append(MP)
            player_names.append(stats['name'] + str(ty))
            player_teams.append(name)
            player_years.append(ty)
            player_heights.append(d['Hgt'])
        except Exception as e:
            print(name,team)
            print(e)
            print(d.keys())
            raise
X.columns

In [ ]:
#all_tables[2018]['GSW']

In [ ]:
R1,R2 = [],[]
for p_v in player_vectors:
    if len(p_v) != 31:
        R1.append(p_v)
    else:
        R2.append(p_v)

R1 = np.array(R1)
R2 = np.array(R2)
Rs = {True:R1,False:R2}
USE_OLD = [len(_)!=31 for _ in player_vectors]

In [ ]:
Rt = {}
for t in [True,False]:
    Xn = Rs[t]
    print(t,Xn.shape)
    scalerX2 = preprocessing.RobustScaler(quantile_range=(30.0, 70.0))
    scalerX2.fit(Xn)
    Xn_s =scalerX2.transform(np.nan_to_num(Xn))
    Xn_fs = fexp[t].transform(np.nan_to_num(Xn_s))
    predict = clfs[t].predict(Xn_fs)
    Rt[t] = np.nan_to_num(ys[t].inverse_transform(predict))

In [ ]:
new_team = [
        ('PHX','PHO'),
        ('BKN','BRK'),
        ('CHA','CHO'),
        ('NOP','NOH'),
        ('NOH','NOK'),
        ('BRK','NJN'),
        ('OKC','SEA'),
        ('MEM','VAN'),
        ('WAS','WSB'),
        ('SAC','KCK'),
        ('LAC','SDC'),
        ('UTA','NOJ'),
        ('SDC','BUF'),
        ('NJN','NYN'),
        ('NYN','NYA'),
        ('NYA','NJA'),
        ('GSW','SFW'),
        ('SAS','DLC'),
        ('DLC','TEX'),
        ('HOU','SDR'),
        ('CHA','CHH'),
        ('SAS','SAA'),
        ('SAA','TEX'),
        ('DEN','DNA'),
        ('DNA','DNR'),
        ('WSB','CAP'),
        ('CAP','BAL'),
        ('BAL','CHZ'),
        ('CHZ','CHP'),
        ('NOH','KEN'),
        ('MIN','SDS'),
        ('SDS','SDA'),
        ('MIA','FLO'),
        ('FLO','MMF'),
        ('MMF','MNM'),
        ('SFW','PHW'),
        ('LAL','MNL'),
        ('UTA','UTS'),
        ('UTS','LAS'),
        ('LAS','ANA'),
        ('CHH','SSL'),
        ('SSL','CAR'),
        ('CAR','HSM'),
        ('DET','FTW'),
        ('ATL','STL'),
        ('STL','MLH'),
        ('MLH','TRI'),
        ('PHI','SYR'),
        ('TOR','PTC'),
        ('PTC','PTP'),
        ('PTP','MNP'),
        ('VAN','MMS'),
        ('MMS','MMT'),
        ('MMT','MMP'),
        ('MMP','NOB'),
        ('ORL','VIR'),
        ('VIR','WSA'),
        ('WSA','OAK'),
        ('SAC','KCK'),
        ('KCK','KCO'),
        ('KCO','KCK'),
        ('KCK','CIN'),
        ('CIN','ROC'),
        ('CHP','BLB'),
        ('IND','INA'),
        ('INA','INO')]
st = set()
for t in new_team:
    st.add(t[0])
    st.add(t[1])

In [ ]:
import random
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    base['version'] = 32

    return players,base
playersOG,base = load_roster('normed.json')
abbrev_to_tid = {_['abbrev']:_['tid'] for _ in base['teams']}
abbrev_to_tid['PHO'] = abbrev_to_tid['PHX']
abbrev_to_tid['CHO'] = abbrev_to_tid['CHA']
abbrev_to_tid['BRK'] = abbrev_to_tid['BKN']
abbrev_to_tid['NOH'] = abbrev_to_tid['NOP']
abbrev_to_tid['NJN'] = abbrev_to_tid['BKN']
abbrev_to_tid['SEA'] = abbrev_to_tid['OKC']
abbrev_to_tid['KCK'] = abbrev_to_tid['SAC']
abbrev_to_tid['SDC'] = abbrev_to_tid['LAC']
abbrev_to_tid['WSB'] = abbrev_to_tid['WAS']
abbrev_to_tid['CHH'] = abbrev_to_tid['CHA']
abbrev_to_tid['VAN'] = abbrev_to_tid['MEM']
abbrev_to_tid['NOK'] = abbrev_to_tid['NOP']
abbrev_to_tid['NOK'] = abbrev_to_tid['NOP']
abbrev_to_tid['BUF'] = abbrev_to_tid['LAC']
abbrev_to_tid['CAP'] = abbrev_to_tid['WAS']

for nt,ot in new_team:
    try:
        abbrev_to_tid[ot] = abbrev_to_tid[nt]
    except:
        print(nt,ot)

base['startingSeason'] = 2019
valid_tids = {abbrev_to_tid[_] for _ in all_tables[2019].keys()}

valid_teams = []
for t in base['teams']:
    if t['tid'] in valid_tids:
        tid = len(valid_teams)
        otid = t['tid']
        t['tid'] = tid
        for k,v in abbrev_to_tid.items():
            if v == otid:
                abbrev_to_tid[k] = tid
        valid_teams.append(t)

base['teams'] = valid_teams


del base['draftPicks']

y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
import random
players = []
rcs = {True:0,False:0}
nc2 = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])/38
team_ocs = defaultdict(list)
for name,use_old,pt in zip(player_names,USE_OLD,player_teams):
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])
    
    
    r_vec = {k: Rt[use_old][rcs[use_old]][y_keys.index(km)] for k,km in y_map.items()}
    r_vec = {k: int(np.clip(v,0,100)) for k,v in r_vec.items()}
    ovr = np.clip(Rt[use_old][rcs[use_old]],0,100) @ nc2
    new_player['ovr'] = ovr
    team_ocs[abbrev_to_tid[pt]].append(ovr)

    rcs[use_old]+=1
    new_player['ratings'] = [r_vec]
    new_player['born'] = {'year':2019-25,'loc':''}

    new_player['tid'] = abbrev_to_tid[pt]

    #    tp += 1
    #else:
    #    new_player['tid'] = -2
    #    year = np.random.randint(1,11)
    #    new_player['born']['year'] += year
    #    new_player['draft'] = {'year':2018+year,"round": 0, "pick": 0, "tid": -1, "originalTid": -1}
    players.append(new_player)



In [ ]:
team_ocs = {k:sorted(v,reverse=True)[13] for k,v in team_ocs.items()}

In [ ]:
for player in players:
    if player['ovr'] < team_ocs[player['tid']]:
        player['tid'] = -1
    del player['ovr']

In [ ]:
base['players'] = players
with open('player_teams.json'.format(tyear[0]),'wt') as fp:
    json.dump(base,fp, sort_keys=True)

In [ ]:
rcs